In [1]:
import yaml
from pathlib import Path
from ollama import Client
import json

# Initialize Ollama client
client = Client(host='http://localhost:11434')

# Load the prompt template
prompt_path = Path('../prompts') / 'entity_extraction.yaml'
with open(prompt_path, 'r', encoding='utf-8') as file:
    prompt_content = yaml.safe_load(file)
prompt = prompt_content['entity_extraction']

# Example text to process
sample_text = """
Apple announced its new iPhone 15 on September 12, 2023. 
Tim Cook presented the event at Apple Park in Cupertino, California. 
The event was also streamed live on YouTube, where millions of viewers tuned in.
"""

# Prepare the messages with both system and user prompts
messages = [
    {
        'role': 'system',
        'content': prompt['system_prompt']
    },
    {
        'role': 'user',
        'content': prompt['user_prompt_template'].format(text_to_process=sample_text)
    }
]

# Call Ollama API
response = client.chat(
    model=prompt['model'],
    messages=messages
)

# Get the raw response content
result = response['message']['content']

## Try to parse as JSON if possible
#try:
#    result = json.loads(result)
#    print(json.dumps(result, indent=2))
#except json.JSONDecodeError:
#    # If JSON parsing fails, print the raw response
#    print("Raw model response:")
#    print(result)

KeyError: '\n  "entities"'